In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re


#initialize empty dataframe
results = pd.DataFrame()

#unique url identifiers for each race
link = [50896,50901,50906,50911, 50916, 50921, 50926]

In [101]:
#lites

#Dealing with names, especially camel case names made this a lot more difficult than I was anticipating
#but I persevered, and I'm glad I did. This is going to be fun to analyze.
html = np.empty(len(link))
df_list = [None] * len(link)

for i,nums in enumerate(link):
    url = 'http://www.promotocross.com/mx/event-results/{}/26'.format(nums)
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html, 'lxml')

#This is where things got incredibly tricky. Names seemed easy, but 
#names like R J and McElrath made a 3 hour ordeal be a 10 hour ordeal.
    elements = soup.find_all("span", class_="athlete-name mobile".split())

    name_gen = "\n".join("{}".format(el.get_text()) for el in elements)
    
#regex to make all camel case standardized lower
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name_gen)
    name_glob = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
#I got a big blob of new line seperated text, so I split to form a list
    name = name_glob.splitlines()
    del name[0::2]
    
    
    #elements = soup.find_all("span", class_="field-content".split())
    #name_rank = "\n".join("{} {}".format(el['class'], el.get_text()) for el in elements)
#Pulling in their race 1 and 2 results. I tried this a bunch of different ways, but I had an epihony that
#made this very simple. I had to split a #-# format, and wasn't sure. Regex can do that really easily, unbeknownst to me...
    elements = soup.find_all("div", class_="field-content".split())
    spread = "\n".join("{}".format(el.get_text()) for el in elements)
    spread = re.findall('\d{1,2}\-\d{1,2}', spread)
    position_list = []
    for nums in spread:
        place = nums.split('-')
        position_list.append(place)

        
#Adding the name of the race to differentiate the data
    #locale = soup.find_all('p')
    locale = ('Location {}'.format(i+1))
    print(locale)
    
    #place_list = place * len(name)
#Splitting the races and making two lists
    race1 = []
    race2 = []
    for element1,element2 in position_list: 
        race1.append(element1)
        race2.append(element2)
#Making a data frame our of all of these
    df = pd.DataFrame({'Name':name, 'Race 1':race1, 'Race 2':race2}).reset_index()
    df['Overall'] = df['index'] + 1
    df.set_index('Name')
    df.columns = [str(col) + ' ' + str(locale) for col in df.columns]
    df_list[i] = df
        


for i,dfs in enumerate(df_list):
#setting up a concatenated dataframe for predicting single race outcome 
    if i == 0:
        dfs.columns = ['Index', 'Name','Race 1', 'Race 2', 'Overall']
        df = dfs
    else:
        dfs.columns = ['Index', 'Name','Race 1', 'Race 2', 'Overall']
        df = pd.concat([df,dfs], axis=0)
        

#setting up a join on name to help us predict expected finish for various riders
    if i == 0:
        df_joined = dfs
               
    else:
        df_joined = df_joined.merge(dfs, left_index=True, right_index = True,how = 'outer')
    
    
    


Location 1
Location 2
Location 3
Location 4
Location 5
Location 6
Location 7


In [102]:
df_joined = df_joined.drop(['Name Location 2','Name Location 3','Name Location 4','Name Location 5','Name Location 6', 'Name Location 7'],1)
df_joined = df_joined.drop(['Index','index Location 2','index Location 3','index Location 4','index Location 5','index Location 6', 'index Location 7'],1)


In [103]:
df_joined

,Index_x,Name_x,Race 1_x,Race 2_x,Overall_x,Index_y,Name_y,Race 1_y,Race 2_y,Overall_y,...,Overall_x,Index_y,Name_y,Race 1_y,Race 2_y,Overall_y,Name,Race 1,Race 2,Overall
0,0,zach _osborne,1,1,1,0.0,zach _osborne,4,2,1.0,...,1.0,0.0,zach _osborne,2,1,1.0,zach _osborne,2,1,1.0
1,1,alex _martin,4,4,2,1.0,jeremy _martin,1,7,2.0,...,2.0,1.0,alex _martin,3,3,2.0,dylan _ferrandis,1,2,2.0
2,2,aaron _plessinger,3,5,3,2.0,dylan _ferrandis,9,1,3.0,...,3.0,2.0,jeremy _martin,1,8,3.0,alex _martin,3,3,3.0
3,3,adam _cianciarulo,2,8,4,3.0,adam _cianciarulo,7,3,4.0,...,4.0,3.0,mitchell _harrison,7,2,4.0,rj _hampshire,5,4,4.0
4,4,austin _forkner,11,2,5,4.0,colt _nichols,5,4,5.0,...,5.0,4.0,dylan _ferrandis,5,4,5.0,jeremy _martin,4,6,5.0
5,5,colt _nichols,5,6,6,5.0,alex _martin,6,5,6.0,...,6.0,5.0,aaron _plessinger,6,11,6.0,joey _savatgy,7,5,6.0
6,6,joey _savatgy,7,7,7,6.0,aaron _plessinger,3,13,7.0,...,7.0,6.0,austin _forkner,4,14,7.0,mitchell _harrison,6,7,7.0
7,7,justin _hill,8,9,8,7.0,michael _mosiman,8,10,8.0,...,8.0,7.0,shane _mc_elrath,9,9,8.0,chase _sexton,9,8,8.0
8,8,mitchell _oldenburg,6,12,9,8.0,joey _savatgy,2,29,9.0,...,9.0,8.0,chase _sexton,12,7,9.0,colt _nichols,8,10,9.0
9,9,mitchell _harrison,9,11,10,9.0,mitchell _harrison,13,8,10.0,...,10.0,9.0,joey _savatgy,11,10,10.0,adam _cianciarulo,15,9,10.0


In [104]:
df

,Index,Name,Race 1,Race 2,Overall
0,0,zach _osborne,1,1,1
1,1,alex _martin,4,4,2
2,2,aaron _plessinger,3,5,3
3,3,adam _cianciarulo,2,8,4
4,4,austin _forkner,11,2,5
5,5,colt _nichols,5,6,6
6,6,joey _savatgy,7,7,7
7,7,justin _hill,8,9,8
8,8,mitchell _oldenburg,6,12,9
9,9,mitchell _harrison,9,11,10
